In [29]:
import numpy as np
import time

In [30]:
def read_input_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    N, D, A, B = map(int, lines[0].split())
    dayoff = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        numbers = list(map(int, lines[i].split()))
        for j in range(0, len(numbers) - 1):
            dayoff[i][numbers[j]] = 1
    return N, D, A, B, dayoff

In [31]:
def initialize_random_solution(N, D, dayoff, A, B):
    X = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        for j in range(1, D+1):
            if dayoff[i][j] == 1:
                X[i][j] = 0
    for i in range(1, N+1):
        for j in range(1, D+1):
            if dayoff[i][j]:
                continue
            if j> 1 and X[i][j - 1] == 4:
                X[i][j] = 0
            else: 
                X[i][j] = np.random.randint(0, 4)
    for d in range(1, D+1):
        for shift in range(1, 5):
            count = np.sum(X[:, d] == shift)
            while count < A: 
                available = [i for i in range(1, N+1) if X[i][d] == 0 and dayoff[i][d] == 0 and X[i][d-1] != 4]
                if not available:
                    break
                i = np.random.choice(available)
                X[i][d] = shift
                count += 1
            while count > B:
                assigned = np.where(X[:, d] == shift)[0]
                if not assigned.size:
                    break
                i = np.random.choice(assigned)
                X[i][d] = 0
                count -= 1
    return X
        


In [32]:
def evaluate_fitness(X, N, D, A, B, dayoff):
    penalty = 0
    # check vi pham ngay nghi
    for i in range(1, N + 1):
        for d in range(1, D +1):
            if dayoff[i][d] and X[i][d] != 0:
                penalty -= 1000

    # check vi pham so luong nguoi lam
    for d in range(1, D +1):
        for shift in range(1, 5):
            count = np.sum(X[:, d] == shift)
            if count < A:
                penalty -= 100 * (A - count)
            elif count > B:
                penalty -= 100 * (count - B)

    # check vi pham ca dem 
    for i in range(1, N + 1):
        for d in range(1, D):
            if X[i][d] == 4 and X[i][d + 1] != 0:
                penalty -= 1000

    max_night_shift = 0
    for i in range(1, N + 1):
        count = np.sum(X[i] == 4)
        if count > max_night_shift:
            max_night_shift = count
    return -max_night_shift + penalty

In [33]:
def get_neighbor(X, N, D, A, B, dayoff):
    X_new = np.copy(X)
    i = np.random.randint(1, N + 1)
    d = np.random.randint(1, D + 1)

    if dayoff[i][d] == 1: 
        return X_new
    if d > 1 and X[i][d - 1] == 4:
        return X_new
    else: 
        new_shift = np.random.randint(0, 5)
        while new_shift == X[i][d]:
            new_shift = np.random.randint(0, 5)
        X_new[i][d] = new_shift
        if new_shift == 4 and d< D:
            X_new[i][d + 1] = 0
            X_new[i][d+1] = 0
    
    #check vi pham ngay nghi
    for i in range(1, N + 1):
        for d in range(1, D + 1):
            if dayoff[i][d] == 1:
                X_new[i][d] = 0

    #check vi pham ca dem
    for i in range(1, N + 1):
        for d in range(1, D):
            if X_new[i][d] == 4 and X_new[i][d + 1] != 0:
                X_new[i][d+1] = 0


    # toi thieu hoa so nguoi lam moi ca
    for i in range(1, D + 1):
        for shift in range(1, 5):
            count = np.sum(X_new[:, i] == shift)
            while A < count < B:
                assigned = np.where(X_new[:, i] == shift)[0]
                if not assigned.size:
                    break
                j = np.random.choice(assigned)
                X_new[j][i] = 0
                count -= 1
    #check vi pham so nguoi lam
    for d in range(1, D + 1):
        for shift in range(1, 5):
            count = np.sum(X_new[:, d] == shift)
            while count < A: 
                available = [i for i in range(1, N + 1) if X_new[i][d] == 0 and dayoff[i][d] == 0 and X_new[i][d-1] != 4]
                if not available:
                    break
                i = np.random.choice(available)
                X_new[i][d] = shift
                count += 1
            while count > B:
                assigned = np.where(X_new[:, d] == shift)[0]
                if not assigned.size:
                    break
                i = np.random.choice(assigned)
                X_new[i][d] = 0
                count -= 1
    for i in range(1, N + 1):
        for d in range(1, D):
            if dayoff[i][d] == 4 and X_new[i][d + 1] != 0:
                X_new[i][d +1] = 0 
    return X_new

In [34]:
def hill_climbing(N, D, A, B, dayoff, max_iter = 1000):
    X = initialize_random_solution(N, D, dayoff, A, B)
    best_fitness = evaluate_fitness(X, N, D, A, B, dayoff)
    best_X = np.copy(X)

    for iteration in range(max_iter):
        X_new = get_neighbor(X, N, D, A, B, dayoff)
        new_fitness = evaluate_fitness(X_new, N, D, A, B, dayoff)

        if new_fitness > best_fitness:
            best_fitness = new_fitness
            best_X = np.copy(X_new)
            X = np.copy(X_new)

    return best_X

In [35]:
if __name__ == "__main__":
    filename = 'Testcase/tc'
    output = 'Output/output'
    list_test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    for tc in list_test:
        start_time = time.time()
        input_file = f"{filename}{tc}.txt"
        output_file = f"{output}{tc}.txt"

        N, D, A, B, dayoff = read_input_from_file(input_file)
        best_solution = hill_climbing(N, D, A, B, dayoff)
        best_fitness = evaluate_fitness(best_solution, N, D, A, B, dayoff)
        with open(output_file, 'w') as file:
            for i in range(1, N + 1):
                file.write(' '.join([str(int(best_solution[i][d])) for d in range(1, D + 1)]) + '\n')
            end_time = time.time()
            file.write("Execution time: {:.2f} seconds\n".format(end_time - start_time))
        print(f"Test case {tc} completed in {end_time - start_time:.2f} seconds.")

Test case 1 completed in 0.99 seconds.
Test case 2 completed in 21.78 seconds.
Test case 3 completed in 75.92 seconds.
Test case 4 completed in 3.78 seconds.
Test case 5 completed in 101.91 seconds.
Test case 6 completed in 190.45 seconds.
Test case 7 completed in 2.70 seconds.
Test case 8 completed in 71.28 seconds.
Test case 9 completed in 13.57 seconds.
Test case 10 completed in 0.95 seconds.
